# Example 4: Commodity futures calendar spread strategy

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/4_Commodities_Calendar_Spread.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Enter your API key
After pasting in your API key, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API key, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install our Python SDK
%pip install sigtech 

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

### 3. Set up your Colab environment

In [ ]:
# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set any parameters 
plt.rcParams['figure.figsize'] = [16, 8]

### 4. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to a calendar spread strategy
A calendar spread strategy is a strategy that involves buying and selling two contracts for the same asset (like a commodity) with the same price but different expiration dates. Typically, a trader will sell a short-term contract and buy a long-term contract and profit from the theta decay.

In the context of commodities, such as in this notebook, the spread reflects the difference between the price of contracts expiring on different dates. The main reasons a trader might want to trade a calendar spread are:

- **Seasonality**: Many commodities have seasonal demand and supply patterns. For example, agricultural commodities depend on planting and harvest cycles. A calendar spread can be used to bet on these seasonal patterns.

- **Cost of Carry**: The price difference between contracts with different expiration dates also reflects the cost of storing and financing the commodity. If this cost is expected to change, a calendar spread could be a good way to trade that expectation.

- **Volatility**: Calendar spreads can be a way to trade changes in volatility. Because longer-term contracts are typically more sensitive to changes in volatility, a trader might use a calendar spread to bet on increasing or decreasing volatility.

- **Lower Risk**: Calendar spreads are typically lower risk than outright long or short positions because they involve both a long and a short position in the same asset. This means that some of the risk is hedged.

- **Income Generation**: If a trader believes that a commodity's price will remain relatively stable, they could use a calendar spread to generate income from the time decay of the near-term contract.

## Our strategy
- We will use the SDK's `RollingFutureStrategy` class to define a function that creates two contract series for a particular commodity.
- We will create a DataFrame of trading signals using the percentage change of the futures over a 21 day period.
- We will use this DataFrame in the SDK's `SignalStrategy` class to simulate the historical performance of a calendar spread strategy which traded based on those signals. 
- We will investigate what the historical performance of this strategy would be using a `BasketStrategy` which rebalances a portfolio of wheat and cocoa. 

## 1. Define our calendar spreads

The following function creates a calendar spread strategy for a particular commodity. Here's what it does step by step:

1. **Define contract aeries**: It first defines two contract series for the same commodity, one for the front contract (`front`) and one for the spot contract (`f0`). Both of these series are defined using the `RollingFutureStrategy` class from the SigTech API. 

   - The `front` series uses the `front` rolling rule, which means it uses the front month contract (the contract that is next to expire). The front offset is set to `-2:-1`, meaning the contract is rolled from the second month to the first month.

   - The `f0` series uses the `f_0` rolling rule, which means it uses the spot month contract (the contract that is currently being delivered). The monthly roll days are set to `5:9`, which means the contract is rolled over the 5th to the 9th day of each month. 

2. **Calculate returns and create signals**: It then calculates the percent change over a 21-day lookback period for both contract series, which represent the returns for these contracts. It drops the `NA` values that occur because of the lookback period.

   - It computes the `signal` by taking the sign of the difference between the `front` and `f0` returns. This creates a series of -1, 0, and 1 values which represent the trading signal for the strategy. If the `front` return is greater than the `f0` return, the signal is 1; if the `front` return is less than the `f0` return, the signal is -1; and if they are equal, the signal is 0. 

3. **Create a `SignalStrategy`**: Finally, it creates a `SignalStrategy` using the SigTech API. This strategy takes `signal_df` as input and trades based on these signals. The strategy is rebalanced weekly on Fridays.

So why is this being used as a signal? The signal is essentially a bet on the price difference between the front month contract and the spot month contract. If the return on the front month contract is higher, the strategy goes long (buys); if the return on the spot month contract is higher, the strategy goes short (sells). This is based on the assumption that these price differences will mean revert or trend, depending on market conditions and the characteristics of the commodity. It's a way to potentially profit from changes in the term structure of futures contracts.

>Tip!\
>Remember if you need more information about a particular class in our SDK you can view its docstring using `?`, try `sig.RollingFutureStrategy?` or `sig.SignalStrategy?`

In [ ]:
def create_calendar_spread(code, lookback):
    front = sig.RollingFutureStrategy(
        currency='USD',
        start_date='2018-01-10',
        contract_code=code,
        contract_sector='COMDTY',
        rolling_rule='front',
        front_offset="-2:-1"
    ) 

    f0 = sig.RollingFutureStrategy(
        currency='USD',
        start_date='2018-01-10',
        contract_code=code,
        contract_sector='COMDTY',
        rolling_rule='f_0',
        monthly_roll_days="5:9"
    )
    
    df = pd.DataFrame({
        'front':front.history(),
        'f0':f0.history()
    }).pct_change(21).dropna()
    
    df['signal']=np.sign(df['front']-df['f0'])
    
    signal_df = pd.DataFrame({
        front.name:df['signal'] ,
        f0.name:df['signal'] * -1
    })
    
    return sig.SignalStrategy(
        currency='USD',
        signal_input=signal_df,
        start_date=signal_df.first_valid_index().date(),
        rebalance_frequency='1W-FRI',
    )

We then define a list of commodities for which we want to create calendar spread strategies.

In [ ]:
commodities_universe = ['W', 'CC'] # W represents Wheat, CC represents Cocoa, see https://sigtechapi.streamlit.app/ for more information.

## 2. Backtest and analyze the calendar spreads

Now that everything is set up, we create calendar spread strategies for all commodities in our universe.

In [ ]:
calendar_spread_strategies = [create_calendar_spread(x, 21).name for x in commodities_universe]

history_df = pd.DataFrame({
    x:create_calendar_spread(x, 21).history() for x in commodities_universe
})

history_df.tail(10)

Next, we plot the historical performance of each calendar spread strategy.

In [ ]:
history_df.plot()

## 3. Backtest the performance a basket of the two calendar spreads 

We combine all individual commodity strategies into a single basket strategy. This strategy will rebalance the commodities in the basket every week on a Friday so that each commodity is represented equally. 

In [ ]:
combined_basket =sig.BasketStrategy(
    currency = 'USD',
    start_date = dtm.date(2018,2,9),
    constituent_names = calendar_spread_strategies,
    rebalance_frequency = '1W-FRI',
    weights = [1/len(calendar_spread_strategies)] * len(calendar_spread_strategies)
)

Finally, we plot the historical performance of our combined basket strategy.

In [ ]:
combined_basket.history().plot()

## Try this strategy for yourself!

Test what the effect of changing one or more of the following parameters is on the basket's total returns:
- Change the number in `.pct_change(21)` or change the method for generating signals entirely.
- Change the commodities being used or add more commodities. See the [SigTech API data catalog](https://sigtechapi.streamlit.app/) for a database of all available commodities.
- Change the `rebalance_frequency` of the strategy. See [Rebalance frequencies](https://learn.sigtech.com/docs/rebalance_frequencies) for all possible values.